In [16]:
# Mount to google drive where data is stored
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [17]:
# If you encounter any problems visit the apache.org link and ensure you have selected the right tgz file
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q https://www.apache.org/dist/spark/spark-3.2.4/spark-3.2.4-bin-hadoop2.7.tgz
!tar xf spark-3.2.4-bin-hadoop2.7.tgz


shell-init: error retrieving current directory: getcwd: cannot access parent directories: Transport endpoint is not connected
shell-init: error retrieving current directory: getcwd: cannot access parent directories: Transport endpoint is not connected
pathconf: Transport endpoint is not connected
pathconf: Transport endpoint is not connected
shell-init: error retrieving current directory: getcwd: cannot access parent directories: Transport endpoint is not connected
tar: spark-3.2.4-bin-hadoop2.7.tgz: Cannot open: Transport endpoint is not connected
tar: Error is not recoverable: exiting now


In [ ]:
# Setting the appropriate paths for Spark
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.2.4-bin-hadoop2.7"

!pip install -q findspark
import findspark
findspark.init()

In [19]:
# Import libraries
from pyspark.sql.functions import *
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, lit, to_timestamp
from pyspark.sql import Window
import pyspark.sql.functions as f
from pyspark.sql.types import StringType
import pandas as pd

In [20]:
# Change directory
import os
os.chdir("/content/gdrive/MyDrive/CSP 572/")

In [21]:
# Create a spark session
spark = SparkSession.builder\
        .master("local")\
        .appName("Colab")\
        .config('spark.ui.port', '4050')\
        .getOrCreate()


In [22]:
# Read 2022 data
data = spark.read.csv("2022.rpt", header=None, sep="|")

In [23]:
# Read the header file
header = pd.read_csv("Header.rpt", sep="|")
cols = header.columns
data = data.toDF(*cols)


In [24]:
# Read all other data files
data_2021 = spark.read.csv("2021.rpt", header=None, sep="|")
data_2020 = spark.read.csv("2020.rpt", header=None, sep="|")
data_2019 = spark.read.csv("2019.rpt", header=None, sep="|")
data_2018 = spark.read.csv("2018.rpt", header=None, sep="|")
data_2017 = spark.read.csv("2017.rpt", header=None, sep="|")

In [25]:
# Combine all data
data = data.union(data_2021)
data = data.union(data_2020)
data = data.union(data_2019)
data = data.union(data_2018)
data = data.union(data_2017)

In [52]:
# Numeric columns list - add to this as you see fit
numeric_cols = ["AdjustedAmountDue1", "TaxAmountDue1", "InterestAmountDue1", "CostAmountDue1", "TotalAmountDue1", "OriginalTaxDue2",
        "AdjustedTaxDue2", "TaxAmountDue2", "InterestAmountDue2", "CostAmountDue2", "TotalAmountDue2", "AssessedValuation",
        "EqualizedEvaluation", "TaxRate", "LastPaymentReceivedAmount1", "LastPaymentReceivedAmount2"
        ] + list(data.columns)[70:80]

In [53]:
# Convert to numeric columns
for col_name in numeric_cols:
    data = data.withColumn(col_name, col(col_name).cast('float'))

In [54]:
# Persist data in disk/memory
# Until this point Spark does not actually execute any of the above statements (Lazy loading). It only creates a plan
# Persist does not get executed until an 'action' command is called on the dataframe
data1 = data.persist()

In [57]:
# The below action with persist called above will ensure that all the steps until data.persist() is run and stored in memory/disk
# Only the first run takes approx 10 mins. If you run it again, it will only take 3 secs as the data is now stored in memory/disk
# It helps in using the modified dataframe directly
# If you do not persist, Spark is going to run all the steps right from read_csv whenever you call an action
print([data1.count(), len(data1.columns)])

[11191850, 81]


In [ ]:
data1.dtypes

In [ ]:
tot = data1.count()

In [44]:
var = "PastTaxSaleStatus"
data1.groupBy(var).agg(count(var), count(var)/tot).show()

+-----------------+------------------------+-------------------------------------+
|PastTaxSaleStatus|count(PastTaxSaleStatus)|(count(PastTaxSaleStatus) / 11191850)|
+-----------------+------------------------+-------------------------------------+
|                Y|                  272729|                  0.02436853603291681|
|                N|                10919119|                   0.9756312852656174|
|             null|                       0|                                  0.0|
+-----------------+------------------------+-------------------------------------+



In [50]:
cols = list(data1.columns)[60:81]
data1.where(data1.AdjustedAmountDue1>0).select(*cols).show()

+-----------+------------------+--------------------------+-----------------+------------+--------------+------------+--------------------------+--------------------------+---------+----------------+---------------------------+-----------------------------+--------------------------+--------------------------------+---------------------+------------------+------------------------+------------------------------+-------------+----------------------+
|RecordCount|CondemnationStatus|MunicipalAcquisitionStatus|AcquisitionStatus|ExemptStatus|BankruptStatus|RefundStatus|LastPaymentReceivedAmount1|LastPaymentReceivedAmount2|EndMarker|TaxDueEstimated1|ReturningVetExemptionAmount|DisabledPersonExemptionAmount|DisabledVetExemptionAmount|DisabledPersonVetExemptionAmount|HomeownerExemptAmount|SeniorExemptAmount|SeniorFreezeExemptAmount|LongtimeHomeownersExemptAmount|VeteranExempt|AdjustedAmountDue1_Org|
+-----------+------------------+--------------------------+-----------------+------------+------

In [ ]:
var = "TaxType"
data.groupby(var)["SegmentCode"].count().reset_index().sort_values("SegmentCode", ascending=False)#.head(50)


,TaxType,SegmentCode
0,0,1864306
1,1,2042


In [ ]:

90000/data.shape[0]

0.04822251798699921

In [43]:
len(data1.columns)

81